In [1]:
import gc
import time
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt
from logzero import logger

In [2]:
def do_count(df, group_cols, agg_type='uint32'):
    agg_name = "_".join(group_cols) + "_count"
    print("Aggregating by ", group_cols, '...')
    gp = df[group_cols][group_cols].groupby(group_cols).size().rename(agg_name).to_frame().reset_index()
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    print(agg_name + " max value = ", df[agg_name].max())
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return(df)


def do_countuniq(df, group_cols, counted, agg_type='uint32'):
    agg_name = "_".join(group_cols) + "_{}_countuniq".format(counted)
    print("Counting unqiue ", counted, " by ", group_cols, '...')
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].nunique().reset_index().rename(columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    print(agg_name + " max value = ", df[agg_name].max())
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return(df)


def do_cumcount(df, group_cols, counted, agg_type='uint32'):
    agg_name = "_".join(group_cols) + "_cumcount"
    print("Cumulative count by ", group_cols, '...')
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].cumcount()
    df[agg_name] = gp.values
    del gp
    print(agg_name + " max value = ", df[agg_name].max())
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return(df)


def do_mean(df, group_cols, counted, agg_type='float32'):
    agg_name = "_".join(group_cols) + "_mean"
    print("Calculating mean of ", counted, " by ", group_cols, '...')
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].mean(
    ).reset_index().rename(columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    print(agg_name + " max value = ", df[agg_name].max())
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return(df)


def do_var(df, group_cols, counted, agg_type='float32'):
    agg_name = "_".join(group_cols) + "_var"
    print("Calculating variance of ", counted, " by ", group_cols, '...')
    gp = df[group_cols + [counted]].groupby(group_cols)[counted].var(
    ).reset_index().rename(columns={counted: agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    print(agg_name + " max value = ", df[agg_name].max())
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return(df)

In [3]:
%%time
df = pd.read_hdf("../data/merged_click_data.hdf", "merged_click_data")

CPU times: user 1.56 s, sys: 9.31 s, total: 10.9 s
Wall time: 5min 5s


In [4]:
df.dtypes

app                                              uint16
channel                                          uint16
click_id                                        float64
device                                           uint16
ip                                               uint32
is_attributed                                   float64
os                                               uint16
click_count_by_ip_os_device_dow                  uint32
click_count_by_ip_os_device_dow_hour             uint16
click_count_by_ip                                uint32
dow                                               uint8
hour                                              uint8
minute                                            uint8
second                                            uint8
rank_by_ip                                       uint32
rank_by_ip_os_device                             uint32
rank_by_ip_os_device_dow                         uint32
click_time_interval_by_ip                       

In [ ]:
%%time
df = do_cumcount(df, ['ip', 'device', 'os'], 'app')
gc.collect()
df = do_cumcount(df, ['ip'], 'os')
gc.collect()
df = do_countuniq(df, ['ip'], 'channel', 'uint8')
gc.collect()
df = do_countuniq(df, ['ip', 'dow'], 'hour', 'uint8')
gc.collect()
df = do_countuniq(df, ['ip'], 'app', 'uint8')
gc.collect()
df = do_countuniq(df, ['ip', 'app'], 'os', 'uint8')
gc.collect()
df = do_countuniq(df, ['ip'], 'device', 'uint16')
gc.collect()
df = do_countuniq(df, ['app'], 'channel')
gc.collect()
df = do_countuniq(df, ['ip', 'device', 'os'], 'app')
gc.collect()
df = do_count(df, ['ip', 'dow', 'hour'])
gc.collect()
df = do_count(df, ['ip', 'app'])
gc.collect()
df = do_count(df, ['ip', 'app', 'os'], 'uint16')
gc.collect()
df = do_var(df, ['ip', 'dow', 'channel'], 'hour')
gc.collect()
df = do_var(df, ['ip', 'app', 'os'], 'hour')
gc.collect()
df = do_var(df, ['ip', 'app', 'channel'], 'dow')
gc.collect()
df = do_mean(df, ['ip', 'app', 'channel'], 'hour')
gc.collect()

Cumulative count by  ['ip', 'device', 'os'] ...
ip_device_os_cumcount max value =  282426
Cumulative count by  ['ip'] ...
ip_cumcount max value =  1421255
Counting unqiue  channel  by  ['ip'] ...
ip_dow_hour_countuniq max value =  24
Counting unqiue  app  by  ['ip'] ...
ip_app_countuniq max value =  277
Counting unqiue  os  by  ['ip', 'app'] ...
ip_app_os_countuniq max value =  148
Counting unqiue  device  by  ['ip'] ...
ip_device_countuniq max value =  551
Counting unqiue  channel  by  ['app'] ...
app_channel_countuniq max value =  49
Counting unqiue  app  by  ['ip', 'device', 'os'] ...
ip_device_os_app_countuniq max value =  100
Aggregating by  ['ip', 'dow', 'hour'] ...
ip_dow_hour_count max value =  44259
Aggregating by  ['ip', 'app'] ...
ip_app_count max value =  220743
Aggregating by  ['ip', 'app', 'os'] ...
ip_app_os_count max value =  55159
Calculating variance of  hour  by  ['ip', 'dow', 'channel'] ...


In [7]:
df.columns

Index(['app', 'channel', 'click_id', 'device', 'ip', 'is_attributed', 'os',
       'click_count_by_ip_os_device_dow',
       'click_count_by_ip_os_device_dow_hour', 'click_count_by_ip', 'dow',
       'hour', 'minute', 'second', 'rank_by_ip', 'rank_by_ip_os_device',
       'rank_by_ip_os_device_dow', 'click_time_interval_by_ip',
       'click_time_interval_by_ip_os_device',
       'click_time_interval_by_ip_os_device_dow',
       'click_time_interval_by_ip_os_device_dow_hour', 'ip_device_os_cumcount',
       'ip_cumcount', 'ip_channel_countuniq', 'ip_dow_hour_countuniq',
       'ip_app_countuniq', 'ip_app_os_countuniq', 'ip_device_countuniq',
       'app_channel_countuniq', 'ip_device_os_app_countuniq',
       'ip_dow_hour_count', 'ip_app_count', 'ip_app_os_count',
       'ip_dow_channel_var', 'ip_app_os_var', 'ip_app_channel_var',
       'ip_app_channel_mean'],
      dtype='object')

In [8]:
df

,app,channel,click_id,device,ip,is_attributed,os,click_count_by_ip_os_device_dow,click_count_by_ip_os_device_dow_hour,click_count_by_ip,...,ip_device_countuniq,app_channel_countuniq,ip_device_os_app_countuniq,ip_dow_hour_count,ip_app_count,ip_app_os_count,ip_dow_channel_var,ip_app_os_var,ip_app_channel_var,ip_app_channel_mean
0,3,379,NaN,1,83230,0.0,13,434,1,28085,...,26,49,49,1,5759,1431,7.893333,36.597950,1.389350,8.521276
1,3,379,NaN,1,17357,0.0,19,183,1,26234,...,22,49,48,1,5245,1451,9.618462,25.959045,1.103880,8.236263
2,3,379,NaN,1,35810,0.0,13,34,1,11002,...,16,49,41,1,2156,462,15.600000,33.348476,1.012026,8.872340
3,14,478,NaN,1,45745,0.0,13,1501,1,188741,...,99,40,58,1,10547,2186,5.947712,36.870251,1.070305,10.361702
4,3,379,NaN,1,161007,0.0,13,28,1,1171,...,5,49,28,1,232,80,10.800000,37.221359,0.619048,11.428572
5,3,379,NaN,1,18787,0.0,16,1,1,4029,...,2,49,20,1,884,30,2.333333,18.975863,1.937909,10.277778
6,3,379,NaN,1,103022,0.0,23,1,1,6099,...,11,49,4,1,1256,3,1.000000,44.333332,0.976950,11.208333
7,3,379,NaN,1,114221,0.0,19,51,1,2334,...,4,49,29,1,404,88,32.000000,29.235109,2.028571,9.466666
8,3,379,NaN,1,165970,0.0,13,19,1,2014,...,6,49,22,1,434,46,8.000000,26.908213,0.566667,14.333333
9,64,459,NaN,1,74544,0.0,22,5,1,3882,...,6,3,18,1,32,1,13.809524,NaN,0.555444,11.875000


In [ ]:
%%time
df.to_hdf("../data/merged_click_data.hdf", "merged_click_data")

In [2]:
%%time
df = pd.read_hdf("../data/merged_click_data.hdf", "merged_click_data")

CPU times: user 1.82 s, sys: 14.5 s, total: 16.3 s
Wall time: 7min 41s


In [3]:
df.dtypes

app                                              uint16
channel                                          uint16
click_id                                        float64
device                                           uint16
ip                                               uint32
is_attributed                                   float64
os                                               uint16
click_count_by_ip_os_device_dow                  uint32
click_count_by_ip_os_device_dow_hour             uint16
click_count_by_ip                                uint32
dow                                               uint8
hour                                              uint8
minute                                            uint8
second                                            uint8
rank_by_ip                                       uint32
rank_by_ip_os_device                             uint32
rank_by_ip_os_device_dow                         uint32
click_time_interval_by_ip                       

In [12]:
%%time
MOD = int(1e9+7)
df["ip_os_device_app_hash"] = (df["ip"].astype(str)+"_"+df["os"].astype(str)+"_"+df["device"].astype(str)+"_"+df["app"].astype(str)).apply(hash)%MOD

CPU times: user 12min 2s, sys: 1min 11s, total: 13min 13s
Wall time: 13min 12s


In [16]:
df["ip_os_device_app_hash"] = df["ip_os_device_app_hash"].astype("uint32")

In [17]:
df.to_hdf("../data/merged_click_data.hdf", "merged_click_data")

# Add time difference features

In [2]:
df = pd.read_hdf("../data/merged_click_data.hdf", "merged_click_data")

In [3]:
df.dtypes

app                                              uint16
channel                                          uint16
click_id                                        float64
device                                           uint16
ip                                               uint32
is_attributed                                   float64
os                                               uint16
click_count_by_ip_os_device_dow                  uint32
click_count_by_ip_os_device_dow_hour             uint16
click_count_by_ip                                uint32
dow                                               uint8
hour                                              uint8
minute                                            uint8
second                                            uint8
rank_by_ip                                       uint32
rank_by_ip_os_device                             uint32
rank_by_ip_os_device_dow                         uint32
click_time_interval_by_ip                       

In [4]:
df = df[["dow", "hour", "minute", "second", "ip_os_device_app_hash"]]

In [9]:
df["click_time_second"] = df["dow"].astype("uint32")*24*60*60 + df["hour"].astype("uint32")*60*60 + df["minute"].astype("uint32")*60 + df["second"].astype("uint32")

In [11]:
df

,dow,hour,minute,second,ip_os_device_app_hash,click_time_second
0,0,14,32,21,525163057,52341
1,0,14,33,34,116105055,52414
2,0,14,34,12,4835573,52452
3,0,14,34,52,271103479,52492
4,0,14,35,8,383355941,52508
5,0,14,36,26,265346367,52586
6,0,14,37,44,451576942,52664
7,0,14,37,59,48712048,52679
8,0,14,38,10,409149612,52690
9,0,14,38,23,991191953,52703


In [12]:
%%time
df.sort_values(by=["ip_os_device_app_hash", "click_time_second"], inplace=True)

CPU times: user 4min 34s, sys: 12.2 s, total: 4min 46s
Wall time: 4min 46s


In [14]:
df

,dow,hour,minute,second,ip_os_device_app_hash,click_time_second
121638,0,16,2,13,26,57733
1787841,0,16,43,25,26,60205
9261562,0,23,58,58,26,86338
11126756,1,0,30,26,26,88226
13377758,1,1,7,58,26,90478
13841862,1,1,16,21,26,90981
13860275,1,1,16,40,26,91000
42169678,1,10,21,31,26,123691
51061818,1,13,11,33,26,133893
52111160,1,13,31,21,26,135081


In [15]:
df["click_time_shift"] = df["click_time_second"].shift()

In [16]:
df

,dow,hour,minute,second,ip_os_device_app_hash,click_time_second,click_time_shift
121638,0,16,2,13,26,57733,NaN
1787841,0,16,43,25,26,60205,57733.0
9261562,0,23,58,58,26,86338,60205.0
11126756,1,0,30,26,26,88226,86338.0
13377758,1,1,7,58,26,90478,88226.0
13841862,1,1,16,21,26,90981,90478.0
13860275,1,1,16,40,26,91000,90981.0
42169678,1,10,21,31,26,123691,91000.0
51061818,1,13,11,33,26,133893,123691.0
52111160,1,13,31,21,26,135081,133893.0


In [17]:
df["click_time_interval_ip_device_os_app_hash"] = df["click_time_second"] - df["click_time_shift"]

In [19]:
df["ip_os_device_app_hash_shift"] = df["ip_os_device_app_hash"].shift()

In [22]:
%%time
df["click_time_interval_ip_device_os_app_hash"].where(df["ip_os_device_app_hash"]==df["ip_os_device_app_hash_shift"], np.nan, inplace=True)

CPU times: user 2.03 s, sys: 508 ms, total: 2.54 s
Wall time: 2.54 s


In [23]:
df

,dow,hour,minute,second,ip_os_device_app_hash,click_time_second,click_time_shift,click_time_interval_ip_device_os_app_hash,ip_os_device_app_hash_shift
121638,0,16,2,13,26,57733,NaN,NaN,NaN
1787841,0,16,43,25,26,60205,57733.0,2472.0,2.600000e+01
9261562,0,23,58,58,26,86338,60205.0,26133.0,2.600000e+01
11126756,1,0,30,26,26,88226,86338.0,1888.0,2.600000e+01
13377758,1,1,7,58,26,90478,88226.0,2252.0,2.600000e+01
13841862,1,1,16,21,26,90981,90478.0,503.0,2.600000e+01
13860275,1,1,16,40,26,91000,90981.0,19.0,2.600000e+01
42169678,1,10,21,31,26,123691,91000.0,32691.0,2.600000e+01
51061818,1,13,11,33,26,133893,123691.0,10202.0,2.600000e+01
52111160,1,13,31,21,26,135081,133893.0,1188.0,2.600000e+01


In [25]:
df["first_click_interval_ip_device_os_app_hash"] = df["click_time_interval_ip_device_os_app_hash"].isnull()

In [27]:
df

,dow,hour,minute,second,ip_os_device_app_hash,click_time_second,click_time_shift,click_time_interval_ip_device_os_app_hash,ip_os_device_app_hash_shift,first_click_interval_ip_device_os_app_hash
121638,0,16,2,13,26,57733,NaN,NaN,NaN,True
1787841,0,16,43,25,26,60205,57733.0,2472.0,2.600000e+01,False
9261562,0,23,58,58,26,86338,60205.0,26133.0,2.600000e+01,False
11126756,1,0,30,26,26,88226,86338.0,1888.0,2.600000e+01,False
13377758,1,1,7,58,26,90478,88226.0,2252.0,2.600000e+01,False
13841862,1,1,16,21,26,90981,90478.0,503.0,2.600000e+01,False
13860275,1,1,16,40,26,91000,90981.0,19.0,2.600000e+01,False
42169678,1,10,21,31,26,123691,91000.0,32691.0,2.600000e+01,False
51061818,1,13,11,33,26,133893,123691.0,10202.0,2.600000e+01,False
52111160,1,13,31,21,26,135081,133893.0,1188.0,2.600000e+01,False


In [28]:
df.drop(columns=["ip_os_device_app_hash_shift", "click_time_shift"], inplace=True)

In [29]:
df

,dow,hour,minute,second,ip_os_device_app_hash,click_time_second,click_time_interval_ip_device_os_app_hash,first_click_interval_ip_device_os_app_hash
121638,0,16,2,13,26,57733,NaN,True
1787841,0,16,43,25,26,60205,2472.0,False
9261562,0,23,58,58,26,86338,26133.0,False
11126756,1,0,30,26,26,88226,1888.0,False
13377758,1,1,7,58,26,90478,2252.0,False
13841862,1,1,16,21,26,90981,503.0,False
13860275,1,1,16,40,26,91000,19.0,False
42169678,1,10,21,31,26,123691,32691.0,False
51061818,1,13,11,33,26,133893,10202.0,False
52111160,1,13,31,21,26,135081,1188.0,False


In [30]:
df["click_time_interval_ip_device_os_app_hash"].max()

341856.0

In [31]:
df["click_time_interval_ip_device_os_app_hash"].fillna(1e9+7, inplace=True)

In [34]:
df["click_time_interval_ip_device_os_app_hash"] = df["click_time_interval_ip_device_os_app_hash"].astype("uint32")

In [36]:
df

,dow,hour,minute,second,ip_os_device_app_hash,click_time_second,click_time_interval_ip_device_os_app_hash,first_click_interval_ip_device_os_app_hash
121638,0,16,2,13,26,57733,1000000007,True
1787841,0,16,43,25,26,60205,2472,False
9261562,0,23,58,58,26,86338,26133,False
11126756,1,0,30,26,26,88226,1888,False
13377758,1,1,7,58,26,90478,2252,False
13841862,1,1,16,21,26,90981,503,False
13860275,1,1,16,40,26,91000,19,False
42169678,1,10,21,31,26,123691,32691,False
51061818,1,13,11,33,26,133893,10202,False
52111160,1,13,31,21,26,135081,1188,False


In [37]:
df.sort_index(inplace=True)

In [38]:
df

,dow,hour,minute,second,ip_os_device_app_hash,click_time_second,click_time_interval_ip_device_os_app_hash,first_click_interval_ip_device_os_app_hash
0,0,14,32,21,525163057,52341,1000000007,True
1,0,14,33,34,116105055,52414,1000000007,True
2,0,14,34,12,4835573,52452,1000000007,True
3,0,14,34,52,271103479,52492,1000000007,True
4,0,14,35,8,383355941,52508,1000000007,True
5,0,14,36,26,265346367,52586,1000000007,True
6,0,14,37,44,451576942,52664,1000000007,True
7,0,14,37,59,48712048,52679,1000000007,True
8,0,14,38,10,409149612,52690,1000000007,True
9,0,14,38,23,991191953,52703,1000000007,True


In [39]:
df[["click_time_interval_ip_device_os_app_hash", "first_click_interval_ip_device_os_app_hash"]].to_hdf("../data/click_time_interval_ip_device_os_app_hash.hdf", "click_time_interval_ip_device_os_app_hash")

# Add first click features

In [1]:
import pandas as pd
df = pd.read_hdf("../data/merged_click_data.hdf", "merged_click_data")

In [4]:
df["is_attributed"].fillna(0.0, inplace=True)
df["is_attributed"] = (df["is_attributed"] == 1.0)

In [7]:
df["is_attributed"] = df["is_attributed"].astype("uint8")

In [11]:
df["click_id"].fillna(1e9+7, inplace=True)

In [12]:
df["click_id"] = df["click_id"].astype("uint32")

In [15]:
df.dtypes

app                                              uint16
channel                                          uint16
click_id                                         uint32
device                                           uint16
ip                                               uint32
is_attributed                                     uint8
os                                               uint16
click_count_by_ip_os_device_dow                  uint32
click_count_by_ip_os_device_dow_hour             uint16
click_count_by_ip                                uint32
dow                                               uint8
hour                                              uint8
minute                                            uint8
second                                            uint8
rank_by_ip                                       uint32
rank_by_ip_os_device                             uint32
rank_by_ip_os_device_dow                         uint32
click_time_interval_by_ip                       

In [16]:
df["click_time_interval_by_ip"].fillna(1e9+7, inplace=True)
df["click_time_interval_by_ip"] = df["click_time_interval_by_ip"].astype("uint32")
df["click_time_interval_by_ip_os_device"].fillna(1e9+7, inplace=True)
df["click_time_interval_by_ip_os_device"] = df["click_time_interval_by_ip_os_device"].astype("uint32")
df["click_time_interval_by_ip_os_device_dow"].fillna(1e9+7, inplace=True)
df["click_time_interval_by_ip_os_device_dow"] = df["click_time_interval_by_ip_os_device_dow"].astype("uint32")
df["click_time_interval_by_ip_os_device_dow_hour"].fillna(1e9+7, inplace=True)
df["click_time_interval_by_ip_os_device_dow_hour"] = df["click_time_interval_by_ip_os_device_dow_hour"].astype("uint32")

In [18]:
df.dtypes

app                                              uint16
channel                                          uint16
click_id                                         uint32
device                                           uint16
ip                                               uint32
is_attributed                                     uint8
os                                               uint16
click_count_by_ip_os_device_dow                  uint32
click_count_by_ip_os_device_dow_hour             uint16
click_count_by_ip                                uint32
dow                                               uint8
hour                                              uint8
minute                                            uint8
second                                            uint8
rank_by_ip                                       uint32
rank_by_ip_os_device                             uint32
rank_by_ip_os_device_dow                         uint32
click_time_interval_by_ip                       

In [19]:
df["first_click_by_ip"] = df["click_time_interval_by_ip"]==1e9+7
df["first_click_by_ip_os_device"] = df["click_time_interval_by_ip_os_device"]==1e9+7
df["first_click_by_ip_os_device_dow"] = df["click_time_interval_by_ip_os_device_dow"]==1e9+7
df["first_click_by_ip_os_device_dow_hour"] = df["click_time_interval_by_ip_os_device_dow_hour"]==1e9+7

In [21]:
df.to_hdf("../data/merged_click_data.hdf", "merged_click_data")